# Tutorial

Kai Foerster, Amin Oueslati, Steve Kerr

## Introduction
Policy motivation: many institutions want to use something like ChatGPT but with their own domain knowledge <br>
Explain what a RAG chatbot is   <br>

### Next steps
-

# Setup

* Install dependencies
* Configure an API key for Hugging Face

In [ ]:
# install dependencies
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off
!pip install -qqq sentence_transformers --progress-bar off
!pip install -qqq chromadb --progress-bar off

## Building a chatbot (no RAG)

In [2]:
import re
import os
import warnings
import gc
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain import LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [3]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

device = f'cuda:{cuda.current_device()}' uf cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',

)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=True, device_map="auto"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_id = "tiiuae/falcon-7b-instruct"
conv_model = HuggingFaceHub(
    huggingfacehub_api_token=os.environ['HF_API_TOKEN'],
    repo_id=model_id,
    model_kwargs={"temperature":0.8,"max_length": 1000}
    )

In [ ]:
#from transformers import AutoModelForCausalLM, AutoTokenizer
#from langchain.llms.huggingface_pipeline import HuggingFacePipeline
#from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("gpt2-medium", padding_side="left")
#tokenizer.pad_token = tokenizer.eos_token
#model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
#hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template="""You are a helpful assistant that answers questions of the user.
{human_message}
"""

prompt=PromptTemplate(template=template, input_variables=["human_message"])

In [ ]:
conv_chain = LLMChain(llm=conv_model, prompt=prompt, verbose=True)

In [ ]:
#res=conv_chain.run("what is string theory?")
#print(res)
print(conv_chain.run("I would "))

### Appending last response to follow up question

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=2)

In [ ]:
memory_chain = ConversationChain(llm=conv_model, memory = memory, verbose=True)

In [ ]:
memory_chain.run("What is your name")

In [ ]:
user_message = "whatever is this"
while user_message != "bye":
    user_message = input("You: ")
    print(memory_chain.run(user_message))

### Hallucinations

In [ ]:
print(conv_chain.run("What is so special about LLMChain?"))

### Source knowledging (manual)

In [ ]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

In [ ]:
source_knowledge

In [ ]:
template_with_context="""You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:{source_knowledge}

{human_message}
"""

prompt2=PromptTemplate(template=template_with_context, input_variables=["human_message",  "source_knowledge"])

In [ ]:
print(prompt2.format(human_message="What is a LLMChain?", source_knowledge=source_knowledge))

In [ ]:
context_chain = LLMChain(llm=conv_model, prompt=prompt2, verbose=True)

In [ ]:
print(context_chain.run({

  'source_knowledge': source_knowledge,

  'human_message': "What is LLMChain?"

}))

## RAG
### Create database to store your corpus on

In [ ]:
# load dependencies
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import shutil

In [ ]:
# set params
DATA_PATH = "data/html"
CHROMA_PATH = "chroma_db"
EMBED_MODEL = "all-MiniLM-L6-v2" # Chroma defaults to "sentence-transformers/all-MiniLM-L6-v2"
# alternative: "BAAI/bge-small-en-v1.5"

# Load Documents

In [ ]:
# load docs
def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(DATA_PATH)
len(documents)

In [ ]:
documents[0]

# Embed Documents & Upload to Vector Database

In [ ]:
# define text embedding model
embedding_func = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)

# See https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
# first, clear out current db
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

# initialize Chroma db and save locally
db = Chroma.from_documents(
    documents=documents, embedding=embedding_func, persist_directory=CHROMA_PATH
    )

db.persist()

# print message
print(f"Saved {len(documents)} chunks to {CHROMA_PATH}.")

# Query Vector Database

In [ ]:
# query vector db
query = "What is the purpose of the Federal Acquisition Regulations?"
matching_docs = db.similarity_search_with_relevance_scores(
    query=query,
    k=4, # number of docs to return
    #score_threshold=.5,
    #filter=[{"":""}]
    )

matching_docs

In [ ]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in matching_docs])

In [ ]:
context_text

### Query data from your database based on your prompt

In [ ]:
### adapted version
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def RAG(query_text):
    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=6)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    if len(context_text) > 1000:
        context_text = context_text[:1000]
        print("Warning: Context exceeded 1000 characters, trimming from the end.")

    prompt_template=PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context",  "question"])
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    chain = LLMChain(llm=conv_model, prompt=prompt_template, verbose=True)
    response_text = chain.run({"context": context_text, "question": query})

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

### Parse the augumented prompt into the chatmodel

In [ ]:
RAG("What does the Federal Acquisition Regulations define?")

### Human evaluation of RAG model
Do we wanna add some other evaluation methods here??

In [ ]:
prompt = HumanMessage(
    content="what safety measures were used in the development of llama 2?"
)

res = chat(messages + [prompt])
print(res.content)

In [ ]:
prompt = HumanMessage(
    content=augment_prompt(
        "what safety measures were used in the development of llama 2?"
    )
)

res = chat(messages + [prompt])
print(res.content)

## References

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb <br>
https://github.com/pixegami/langchain-rag-tutorial/tree/main <br>
https://www.youtube.com/watch?v=LhnCsygAvzY <br>
https://www.youtube.com/watch?v=tcqEUSNCn8I <br>
https://www.mlexpert.io/prompt-engineering/chatbot-with-local-llm-using-langchain

I MADE SOME CHANGES HERE